In [1]:
from mqt.yaqs.core.data_structures.networks import MPS, MPO
from mqt.yaqs import simulator
from mqt.yaqs.core.data_structures.simulation_parameters import AnalogSimParams, Observable
from mqt.yaqs.core.libraries.gate_library import XX, Z, ZZ, RuntimeCost
from mqt.yaqs.core.data_structures.noise_model import NoiseModel

import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
def tdvp_simulator(H_0, noise_model, state=None):
    L = H_0.length

    if noise_model.processes[0]["strength"] == 1e-3:
        num_traj = 1
    elif noise_model.processes[0]["strength"] == 1e-2:
        num_traj = 10
    elif noise_model.processes[0]["strength"] in [1e-1, 1e2]:
        num_traj = 10

    max_bond_dim = 2**(L-1)
    min_bond_dim = 2
    threshold = 1e-12
    sample_timesteps = True
    order = 1

    elapsed_time = 5
    dt = 0.1

    state = MPS(length=L, state="zeros")

    # measurements = [Observable(Z(), [site]) for site in range(L)] + [Observable("runtime_cost")] + [Observable("max_bond")] + [Observable("total_bond")]
    measurements = [Observable("entropy", [site, site+1]) for site in range(L-1)] + [Observable("runtime_cost")] + [Observable("max_bond")] + [Observable("total_bond")]
    # measurements = [Observable(ZZ(), [site, site+1]) for site in range(L-1)]
    sim_params = AnalogSimParams(measurements, elapsed_time, dt, num_traj, max_bond_dim, min_bond_dim, threshold, order, sample_timesteps=sample_timesteps)

    simulator.run(state, H_0, sim_params, noise_model=noise_model)

    return sim_params.observables

In [3]:
L = 16
gammas = [1e-3, 1e-2, 1e-1, 1e2]
for j, gamma in enumerate(gammas):
    print(j, "of", len(gammas)-1)
    J = 1
    h = 1
    H_0 = MPO()
    H_0.init_ising(L, J, h)
    # H_0.init_heisenberg(L, J, J, J, h)

    results = []
    # Define the noise model
    single_site = [{"name": name, "sites": [i], "strength": gamma} for i in range(L) for name in ["dephasing", "bitflip", "bitphaseflip"]]
    noise_model = NoiseModel(single_site)

    cost = tdvp_simulator(H_0, noise_model)
    results.append(cost)
    filename = f"correlator_{j}.pickle"
    with open(filename, 'wb') as handle:
        pickle.dump(results, handle)

0 of 3


C:\Users\Aaron\Documents\GitHub\mqt-yaqs\src\mqt\yaqs\core\data_structures\networks.py:313: RuntimeWarning: divide by zero encountered in log
  entropy = -np.sum(s_vec**2 * np.log(s_vec**2))
C:\Users\Aaron\Documents\GitHub\mqt-yaqs\src\mqt\yaqs\core\data_structures\networks.py:313: RuntimeWarning: invalid value encountered in multiply
  entropy = -np.sum(s_vec**2 * np.log(s_vec**2))


1 of 3
2 of 3
3 of 3


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from scipy.interpolate import make_interp_spline

filename = "results_14.pickle"
with open(filename, 'rb') as f:
    results = pickle.load(f)

results = np.array(results).T

# ----- Configure global style -----
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Computer Modern Roman"],
    "axes.labelsize": 14,
    "font.size": 13,
    "axes.titlesize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "xtick.direction": "out",
    "ytick.direction": "out",
    "axes.linewidth": 1.0,
    "legend.fontsize": 12,
    "figure.figsize": (6, 4)
})

# ----- Input data -----
results = np.array(results)
num_curves = len(results)

# Set up figure and axis
fig, ax = plt.subplots()

# Create colormap and normalize
cmap = plt.get_cmap('magma_r')
norm = mcolors.Normalize(vmin=0, vmax=num_curves - 1)
colors = cmap(np.linspace(0, 1, num_curves))
gammas = np.logspace(-3, 3, 30)
# Smooth and plot each curve
for j in range(num_curves):
    x = np.array(gammas)
    y = results[j]
    
    if len(x) >= 4:
        x_smooth = np.logspace(np.log10(x[0]), np.log10(x[-1]), 300)
        spline = make_interp_spline(np.log10(x), y, k=3)
        y_smooth = spline(np.log10(x_smooth))
        ax.plot(x_smooth, y_smooth, color=colors[j], lw=1.5)
    else:
        ax.plot(x, y, color=colors[j], lw=1.5)

# Log scale and labels
ax.set_xscale('log')
# ax.set_yscale('log')
ax.set_xlabel('$\\gamma$ (Depolarizing noise)', labelpad=5)
ax.set_ylabel('Runtime cost', labelpad=5)

# Clean up axes
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add colorbar
sm = cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, pad=0.02)
cbar.set_label('Trotter steps', labelpad=5)
cbar.ax.tick_params(direction='out', length=3)

# Tight layout
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'results_14.pickle'

In [ ]:
# t1_list = []
# t2_list = []
# t3_list = []
# t4_list = []
# t5_list = []
# for j in range(len(results)):
#     t1_list.append(results[j][9])
#     t2_list.append(results[j][19])
#     t3_list.append(results[j][29])
#     t4_list.append(results[j][39])
#     t5_list.append(results[j][49])

# plt.plot(gammas, t1_list, label='T=1')
# plt.plot(gammas, t2_list, label='T=2')
# plt.plot(gammas, t3_list, label='T=3')
# plt.plot(gammas, t4_list, label='T=4')
# plt.plot(gammas, t5_list, label='T=5')
# plt.legend()
# plt.xscale('log')

In [ ]:
filename = "results_no_noise.pickle"
with open(filename, 'rb') as f:
    results = pickle.load(f)

In [ ]:
heatmap = [observable.results for observable in results['exp_vals'][0::-2]]

fig, ax = plt.subplots(1, 1)
im = plt.imshow(heatmap, aspect="auto", extent=(0, 9.9, L, 0))

In [ ]:
filename = "results_gamma_0.pickle"
with open(filename, 'rb') as f:
    results = pickle.load(f)
plt.plot(results['exp_vals'][-1].results)

filename = "results_gamma_1.pickle"
with open(filename, 'rb') as f:
    results = pickle.load(f)
plt.plot(results['exp_vals'][-1].results)

filename = "results_gamma_2.pickle"
with open(filename, 'rb') as f:
    results = pickle.load(f)
plt.plot(results['exp_vals'][-1].results)